Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Image Classification Using Scikit-learn

## Prerequisites

* [Setup Azure Arc-enabled Machine Learning Training and Inferencing on AKS on Azure Stack HCI](https://github.com/Azure/AML-Kubernetes/tree/master/docs/AKS-HCI/AML-ARC-Compute.md)

* [Setup NFS Server on Azure Stack HCI and Use your Data and run managed Machine Learning Experiments On-Premises](https://github.com/Azure/AML-Kubernetes/tree/master/docs/AKS-HCI/Train-AzureArc.md)

* Make sure the NFS Server is mounted on the notebook execution machine. In this notebook, it will upload the training data to NFS Server.

*    Last but not least, you need to be able to run a Notebook. (azureml-core, azureml-opendatasets, numpy, matplotlib, requests are required)

   If you are using an Azure Machine Learning Notebook VM, you are all set. Otherwise, make sure you go through the configuration Notebook located at [here](https://github.com/Azure/MachineLearningNotebooks) first. This sets you up with a working config file that has information on your workspace, subscription id, etc.

## Initialize AzureML workspace

Initialize a [Workspace](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#workspace) object from the existing workspace you created in the Prerequisites step. `Workspace.from_config()` creates a workspace object from the details stored in `config.json`. 

If you haven't done already please go to `config.json` file and fill in your workspace information.

In [ ]:
from azureml.core.workspace import Workspace,  ComputeTarget
from azureml.exceptions import ComputeTargetException

ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep='\n')

## Download mnist data

Perform pip install azureml-opendatasets to get the open dataset package, use this function to download mnist data later. This allows you to avoid download the data again when you run this notebook multiple times. The actual download time may take 2 minutes.

In [ ]:
from azureml.core import Dataset
from azureml.opendatasets import MNIST
import os

def download_mnist_data():
    data_folder = os.path.join(os.getcwd(), 'mnist_data')
    os.makedirs(data_folder, exist_ok=True)

    mnist_file_dataset = MNIST.get_file_dataset()
    path = mnist_file_dataset.download(data_folder, overwrite=True)
    downloaded_folder = os.path.dirname(path[0])
    print("downloaded to", downloaded_folder)
    
    return downloaded_folder

downloaded_folder = download_mnist_data()
downloaded_folder


## Prepare the mnist data to NFS server (Optional)

The above download_mnist_data() function will download four files t10k-images-idx3-ubyte.gz, t10k-labels-idx1-ubyte.gz, train-images-idx3-ubyte.gz and train-labels-idx1-ubyte.gz to downloaded_folder.  Your next step is to copy these files to NFS server.
Run run this step, you need to make sure the NFS path is mounted on the notebook execution machine. Replace `<NFS Mount Point on notebook execution machine>` with the correct path.

This step is optional, you can use other ways, such as directly downloading training data on NFS server to prepare the mnist data.

In [ ]:
nfs_mount_path = "<NFS Mount Point on notebook execution machine>"

import os, shutil
mnist_dir = os.path.join(nfs_mount_path, 'mnist')
shutil.rmtree(mnist_dir, ignore_errors=True)
os.makedirs(mnist_dir, exist_ok=True)

for filename in os.listdir(downloaded_folder):
    filepath = os.path.join(downloaded_folder, filename)
    destpath = os.path.join(mnist_dir, filename)
    print(f"Copying files from {filepath} to {destpath}")
    shutil.copyfile(filepath, destpath)


## Setup compute target

Find the attach name for the Arc enabled AKS-HCI in your AzureML workspace.

attach_name is the attached name for your AKS-HCI cluster you setup in [this step](https://github.com/Azure/AML-Kubernetes/blob/master/docs/AKS-HCI/AML-ARC-Compute.md#attach-your-azure-arc-enabled-cluster-to-your-azure-machine-learning-workspace-as-a-compute-target)

In [ ]:
from azureml.core.compute import KubernetesCompute

attach_name = "<NAME_OF_AML_ATTACHED_COMPUTE_OF_YOUR_ASH_CLUSTER>"
arcK_target = KubernetesCompute(ws, attach_name)

print(f"compute target id in endpoint yaml: azureml:{arcK_target.name}, instance type name in deployment yaml: {arcK_target.default_instance_type}")

## Configure the training job and submit

### Create an experiement

In [ ]:
from azureml.core import Experiment

experiment_name = 'mnist-demo'

exp = Experiment(workspace=ws, name=experiment_name)

### Create an environment

In [ ]:
# customized environment

from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
# to install required packages
env = Environment('tutorial-env')
cd = CondaDependencies.create(pip_packages=['azureml-dataset-runtime[pandas,fuse]', 'azureml-defaults'], conda_packages = ['scikit-learn==0.22.1'])

env.python.conda_dependencies = cd

### Configure the training job

`<MountPathOnTrainingPod>` is the same as the mountPath defined in mount-config.yaml.

In [ ]:
from azureml.core import ScriptRunConfig

data_folder = "<MountPathOnTrainingPod>"+"/mnist" # training data are saved to <mountPoint>/mnist (have to use / as the path separator)

args = ['--data-folder', data_folder, '--regularization', 0.5]
script_folder =  "mnist_script"
src = ScriptRunConfig(source_directory=script_folder,
                      script='train.py', 
                      arguments=args,
                      compute_target=arcK_target,
                      environment=env)

### Submit the job

Run your experiment by submitting your ScriptRunConfig object. Note that this call is asynchronous.

In [ ]:
run = exp.submit(config=src)
run.wait_for_completion(show_output=True)  # specify True for a verbose log

### Register the model

Register the trained model.

In [ ]:
model_name='sklearn_mnist'

In [ ]:
# register model
model = run.register_model(model_name=model_name,
                           model_path='outputs/sklearn_mnist_model.pkl')

The machine learning model named "sklearn_mnist" should be registered in your AzureML workspace.

### Get the model

In [ ]:
from azureml.core.model import Model
model = Model(ws, model_name)
model_id = f"azureml:{model.name}:{model.version}"
print(f"Get {model.name}, latest version {model.version}, id in deployment.yml: {model_id}")

## Deploy and score a machine learning model by using a managed online endpoint

Currently, amlarc inference is only supported with Azure Machine Learning CI (v2). Please follow [doc](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-train-cli?view=azure-devops#prerequisites) to configure the prerequisites. This notebook will invoke Azure Machine Learning CI (v2) directly.

In [ ]:
endpoint = '<sklearn-mnist endpoint name>'

import os
from pathlib import Path
prefix = Path(os.getcwd())
endpoint_file = str(prefix.joinpath("endpoint.yml"))
deployment_file = str(prefix.joinpath("deployment.yml"))
print(f"Using Endpoint file: {endpoint_file}, Deployment file: {deployment_file} please replace <modelId> (e.g. azureml:sklearn_mnist:1), <instanceTypeName> (e.g. defaultInstanceType) and <computeTargetName> (e.g. azureml:amlarc-compute) according above output")

Need to **replace the properties in deployment.yml**, including,
* `<modelId>`: example value: azureml:sklearn_mnist:1
* `<instanceTypeName>`: example value: defaultInstanceType

Need to **replace the properties in endpoint.yml**, including,
* `<computeTargetName>`: example value: azureml:amlarc-compute

In [ ]:
import helpers
from azureml.core.workspace import Workspace

ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep='\n')

In [ ]:
helpers.run(f"az ml online-endpoint create -n {endpoint} -f {endpoint_file} -w {ws.name} -g {ws.resource_group}")

In [ ]:
helpers.run(f"az ml online-endpoint show -n {endpoint} -w {ws.name} -g {ws.resource_group}")

In [ ]:
helpers.run(f"az ml online-deployment create -n blue --endpoint {endpoint} -f {deployment_file} -w {ws.name} -g {ws.resource_group} --all-traffic")

## Test trained model

### Test with inputs

Here you may use the image from test asset. The first 30 images and its labels are shown below.

In [ ]:
from mnist_script.utils import load_data
import os
import glob
import matplotlib.pyplot as plt
import numpy as np

data_folder = os.path.join(os.getcwd(), 'mnist_data')

X_test = load_data(glob.glob(os.path.join(data_folder,"**/t10k-images-idx3-ubyte.gz"), recursive=True)[0], False) / 255.0
y_test = load_data(glob.glob(os.path.join(data_folder,"**/t10k-labels-idx1-ubyte.gz"), recursive=True)[0], True).reshape(-1)

# show first 30 figures

count = 0
sample_size = 30
plt.figure(figsize = (16, 6))
# for i in np.random.permutation(X_test.shape[0])[:sample_size]:
for i in range(30):
    count = count + 1
    plt.subplot(1, sample_size, count)
    plt.axhline('')
    plt.axvline('')
    plt.text(x = 10, y = -10, s = y_test[i], fontsize = 18)
    plt.imshow(X_test[i].reshape(28, 28), cmap = plt.cm.Greys)
plt.show()

Get score_uri and access_token from AZ CLI (Currently only Azure Machine Learning CLI (v2) supported)

In [ ]:
# get score_url and access_token from AZ CLI
import helpers
from azureml.core.workspace import Workspace
ws = Workspace.from_config()
cmd = f"az ml online-endpoint show -n {endpoint} -w {ws.name} -g {ws.resource_group}"
properties = helpers.run(cmd, return_output=True, no_output=True)

cmd = f"az ml online-endpoint get-credentials -n {endpoint} -w {ws.name} -g {ws.resource_group}"
credentials = helpers.run(cmd, return_output=True, no_output=True)

print(f"Got endpoint and credentials.")

Test the second image: 2

In [ ]:
import json
prop_response = json.loads(properties.replace(os.linesep,""))
score_uri = prop_response["scoring_uri"]

cred_response = json.loads(credentials.replace(os.linesep, ""))
access_token = cred_response["accessToken"]

import requests
# second number should be 2
test = json.dumps({"data": X_test.tolist()[1:2]})
headers = {'Content-Type': 'application/json', 'Authorization': f"Bearer {access_token}"}
r = requests.post(score_uri, data=test, headers=headers)
print(f"predictions: {r.json()}")